In [1]:
# from sdRDM.generator import generate_python_api

In [2]:
# generate_python_api("specifications/enzymepynetics.md", "", "EnzymePynetics")

In [2]:
%reload_ext autoreload
%autoreload 2

from EnzymePynetics import Estimator

## Initialize the Estimator from an EnzymeML document

In [3]:
e = Estimator.from_enzymeml(
    enzymeml_doc="/Users/max/Documents/GitHub/MTPHandler/SLAC_kinetic_assay.json",
    measured_reactant="ABTS",
)

abts, abts_radical = e.reactants
slac = e.enzymes[0]

## Define the reaction

In [4]:
oxidation = e.add_reaction(
    id="r1", name="Oxidation", educt=abts, product=abts_radical, enzyme=slac
)

## Add a kinetic models

In [5]:
michaelis = e.add_model(
    id="model1",
    name="michaelis-menten",
    equation="substrate = -substrate * enzyme * k_cat / (K_m + substrate)",
)

enzyme_inactivation = e.add_model(
    id="model2",
    name="enzyme inactivation",
    equation="enzyme = -enzyme * k_ie",
)

competitive_product = e.add_model(
    id="model3",
    name="competitive product inhibition",
    equation="substrate = -substrate * enzyme * k_cat / K_m * (1 + product / K_ic) + substrate",
)

In [48]:
sys2 = e.reaction_systems[1]

In [49]:
sys2.fit(
    e.substrate_data[:, :3],
    e.product_data[:, :3],
    e.enzyme_data[:, :3],
    e.time_data[:, :3]
)

we startin
1.3098478952870691
0.7178072017469643
0.7701635328794881


In [44]:
res = sys2.fit(
    e.substrate_data,
    e.product_data,
    e.enzyme_data,
    e.time_data,
)

we startin
8.420560451942784
0.49187890798916195
0.2189262082552705


In [46]:
sys2.reactions[1].model.parameters

[KineticParameter(id='kineticparameter2', name='k_ie', value=0.2189262082552705, unit='min', initial_value=0.007701635339554948, upper=0.7701635339554948, lower=7.701635339554948e-06, is_global=False, stdev=0.02406250441552234, constant=False, ontology=None)]

In [37]:
res

## Next steps:

- write back results to DM
- write back results to EnzymeML
- visualization

In [10]:
print(competitive_product)

KineticModel
├── id = model3
├── name = competitive product inhibition
├── equation = substrate = -substrate * enzyme * k_cat / K_m * (1 + product / K_ic) + substrate
└── parameters
    ├── 0
    │   └── KineticParameter
    │       ├── id = kineticparameter3
    │       ├── name = K_m
    │       ├── value = nan
    │       ├── unit = umol / l
    │       ├── initial_value = 100.0
    │       ├── upper = 10000.0
    │       ├── lower = 0.1
    │       ├── is_global = False
    │       ├── constant = False
    │       └── ontology = SBO:0000027
    ├── 1
    │   └── KineticParameter
    │       ├── id = kineticparameter4
    │       ├── name = K_ic
    │       ├── value = nan
    │       ├── unit = umol / l
    │       ├── initial_value = 100.0
    │       ├── upper = 10000.0
    │       ├── lower = 0.1
    │       ├── is_global = False
    │       └── constant = False
    └── 2
        └── KineticParameter
            ├── id = kineticparameter5
            ├── name = k_cat
           

In [11]:
from scipy.integrate import odeint
from lmfit import Model, Parameters

In [12]:
m = Model(michaelis.function)
m.param_names
m.independent_vars

['k_cat']

In [13]:
moodels = [michaelis, enzyme_inactivation]

In [14]:
lmfit_params = Parameters()

for model in moodels:
    for param in model.parameters:
        lmfit_params.add(
            name=param.name, value=param.initial_value, min=param.lower, max=param.upper
        )

In [15]:
michaelis.function.__code__.co_varnames

('k_cat', 'substrate', 'K_m', 'enzyme')

In [16]:
lmfit_params

name,value,initial value,min,max,vary
K_m,100.000000,100.0,0.10000000,10000.0000,True
k_cat,8.75670571,8.756705705198392,0.00875671,875.670571,True
k_ie,0.00770164,0.007701635339554948,7.7016e-06,0.77016353,True


In [17]:
enzyme_eq = enzyme_inactivation.function
substrate_eq = michaelis.function

In [18]:
def DS(
    substrate,
    enzyme,
    k_cat,
    K_m,
):
    return -substrate * enzyme * k_cat / (K_m + substrate)


def DE(enzyme, k_ie):
    return -enzyme * k_ie

In [19]:
def model(
    species,
    time,
    params,
):
    species_dict = dict(zip(["substrate", "enzyme", "product"], species))
    params_dict = params.valuesdict()
    combined_dict = species_dict | params_dict

    subtrate_args = substrate_eq.__code__.co_varnames
    substrate_dict = {k: combined_dict[k] for k in subtrate_args}

    enzyme_args = enzyme_eq.__code__.co_varnames
    enzyme_dict = {k: combined_dict[k] for k in enzyme_args}

    d_substrate = substrate_eq(**substrate_dict)
    d_enzyme = enzyme_eq(**enzyme_dict)
    d_product = -d_substrate

    return [d_substrate, d_enzyme, d_product]

In [20]:
def alternate_model(species, time, params):
    params = params.valuesdict()

    substrate, enzyme, product = species

    d_substrate = -substrate * enzyme * params["k_cat"] / (params["K_m"] + substrate)
    d_enzyme = -enzyme * params["k_ie"]

    return np.array([d_substrate, d_enzyme, -d_substrate])

In [21]:
%%timeit
alternate_model(
    [e.substrate_data, e.enzyme_data, e.product_data], e.time_data, lmfit_params
)

NameError: name 'np' is not defined

In [ ]:
%%timeit
model([e.substrate_data, e.enzyme_data, e.product_data], e.time_data, lmfit_params)

1.05 ms ± 5.32 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [ ]:
def simulate(times, init_conditions, params):
    return np.array(
        [
            odeint(
                func=model,
                y0=init_condition,
                t=time,
                args=(params,),
            )
            for init_condition, time in zip(init_conditions, times)
        ]
    )

In [ ]:
np.array([e.substrate_data[:, 0], e.enzyme_data[:, 0], e.product_data[:, 0]]).T[0]

array([nan,  4., nan])

In [ ]:
%%timeit
r = simulate(
    e.time_data,
    np.array([e.substrate_data[:, 0], e.enzyme_data[:, 0], e.product_data[:, 0]]).T,
    lmfit_params,
)



6.64 ms ± 90.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
import numpy as np
from lmfit import Parameters, minimize
from scipy.integrate import odeint

# Define your ODE model function


def ode_model(params, t, y):
    k1 = params["k1"].value
    k2 = params["k2"].value

    dydt = np.zeros_like(y)
    dydt[0] = -k1 * y[0]
    dydt[1] = k1 * y[0] - k2 * y[1]

    return dydt


# Create a Parameters object and set initial guess values
params = Parameters()
params.add("k1", value=0.1)
params.add("k2", value=0.2)

# Define the time points and experimental data
t = np.array([0.0, 1.0, 2.0, 3.0, 4.0])
data = np.array([1.0, 0.5, 0.3, 0.2, 0.1])

# Define the residual function


def residual(params, t, data):
    y0 = [1.0, 0.0]  # Initial conditions
    model_result = odeint(ode_model, y0, t, args=(params,))
    # Extract the second component of the solution
    model_data = model_result[:, 1]
    return model_data - data


# Perform the fitting
result = minimize(residual, params, args=(t, data))

# Print the fitted parameter values
print("Fitted Parameters:")
print(result.params)

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [ ]:
import numpy as np

# Create a NumPy array with shape (3, 12)
arr = np.array(
    [
        [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
        [13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24],
        [25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36],
    ]
)

# Define a function to calculate row sums


def row_sum(row):
    return np.sum(row)


# Use np.vectorize to apply the row_sum function row-wise (axis=1)
vectorized_row_sum = np.vectorize(row_sum, signature="(n)->()")

# Apply the vectorized function to the entire array row-wise
row_sums = vectorized_row_sum(arr)

# Print the row sums
print(row_sums)

[ 78 222 366]


In [ ]:
from sdRDM import DataModel

EnzymeML = DataModel.from_git("https://github.com/EnzymeML/enzymeml-specifications.git")

In [ ]:
from EnzymePynetics.modified.reactionsystem import ReactionSystem

In [ ]:
ReactionSystem(name="sdfadf", reactions=e.reactions)

ReactionSystem(id='reactionsystem1', name='sdfadf', reactions=[Reaction(id='r1', name='Oxidation', reversible=False, temperature=35.0, temperature_unit='C', ph=3.5, ontology='SBO:0000176', uri=None, creator_id=None, model=None, educts=[ReactionElement(id='reactionelement0', species_id='s0', stoichiometry=1.0, constant=False, ontology='SBO:0000015')], products=[ReactionElement(id='reactionelement1', species_id='s2', stoichiometry=1.0, constant=False, ontology='SBO:0000011')], modifiers=[ReactionElement(id='reactionelement2', species_id='p0', stoichiometry=1.0, constant=True, ontology='SBO:0000013')])], result=ModelResult(id='modelresult0', equations=[], parameters=[], fit_success=None, AIC=None, BIC=None, RMSD=None))

In [10]:
import numpy as np

In [19]:
e1 = np.repeat(1, 10).reshape(2, 5)
e2 = np.repeat(2, 10).reshape(2, 5)
e3 = np.repeat(3, 10).reshape(2, 5)


np.zeros(e1.shape)

np.array([e1[:, 0], e2[:, 0], e3[:, 0]]).T

array([[1, 2, 3],
       [1, 2, 3]])